In [1]:
import pandas as pd
import os
import glob
import geopy.distance
import matplotlib.pyplot as plt
import requests
import math
import numpy as np

In [2]:
def read_data(folder): 
  '''
  Pasamos el path del folder donde están los datos dinámicos
  '''
  merged_data = pd.DataFrame() 
  csv_files = glob.glob(os.path.join(folder, "*.csv"))

  for f in csv_files:  
      df = pd.read_csv(f)
      merged_data = pd.concat([merged_data,df])
  return merged_data

def segment_data(data, path_ice, path_ev):
  '''
  Pasamos los datos dinámicos (ya leídos) y los paths a los csv estáticos
  
  '''
  ice_hev = pd.read_csv(path_ice, sep = ';') 
  phev_ev = pd.read_csv(path_ev, sep = ';')
  hev_id = ice_hev.loc[ice_hev['Vehicle Type']=='HEV']
  ice_id = ice_hev.loc[ice_hev['Vehicle Type']=='ICE']
  ev_id = phev_ev.loc[phev_ev['EngineType']=='EV']
  phev_id = phev_ev.loc[phev_ev['EngineType']=='PHEV']
  ev = pd.merge(data,ev_id,on ='VehId', how = 'inner')
  phev = pd.merge(data,phev_id,on ='VehId', how = 'inner')
  hev = pd.merge(data,hev_id,on ='VehId', how = 'inner')
  ice = pd.merge(data,ice_id,on ='VehId', how = 'inner')
  return ev, phev, hev, ice

data = read_data('/content/drive/MyDrive/Monitoría EV/Data')
path_ice, path_ev ='/content/drive/MyDrive/Monitoría EV/Static Data/VED_Static_Data_ICE&HEV.csv','/content/drive/MyDrive/Monitoría EV/Static Data/VED_Static_Data_PHEV&EV.csv'
ev, phev, hev, ice = segment_data(data,path_ice,path_ev)


Limpiar datos carros eléctricos

In [3]:
ev.drop(columns = ['MAF[g/sec]','Engine RPM[RPM]','Absolute Load[%]','Short Term Fuel Trim Bank 1[%]','Short Term Fuel Trim Bank 2[%]', 
                   'Long Term Fuel Trim Bank 1[%]', 'Long Term Fuel Trim Bank 2[%]','Fuel Rate[L/hr]', 'Air Conditioning Power[kW]', 'Transmission',
                   'EngineType', 'Drive Wheels', 'Engine Configuration & Displacement', 'Vehicle Class'], inplace = True)
ev.fillna(method='ffill')

,DayNum,VehId,Trip,Timestamp(ms),Latitude[deg],Longitude[deg],Vehicle Speed[km/h],OAT[DegC],Air Conditioning Power[Watts],Heater Power[Watts],HV Battery Current[A],HV Battery SOC[%],HV Battery Voltage[V],Generalized_Weight
0,1.719774,10,1558,0,42.277066,-83.763404,53.590000,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
1,1.719774,10,1558,200,42.277066,-83.763404,51.980000,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
2,1.719774,10,1558,1200,42.277066,-83.763404,50.369999,5.0,0.0,2250.0,-21.5,96.341469,386.0,3500
3,1.719774,10,1558,1500,42.277066,-83.763404,50.369999,5.0,0.0,2250.0,23.5,96.341469,390.5,3500
4,1.719774,10,1558,2300,42.277066,-83.763404,49.799999,5.0,0.0,2250.0,23.5,96.341469,390.5,3500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476303,208.990283,541,1153,108800,42.285123,-83.801513,66.449997,30.0,0.0,0.0,-13.5,59.024395,376.5,3500
476304,208.990283,541,1153,109600,42.285123,-83.801513,66.400002,30.0,0.0,0.0,-13.5,59.024395,376.5,3500
476305,208.990283,541,1153,109800,42.285123,-83.801513,66.400002,30.0,0.0,0.0,-14.0,59.024395,376.5,3500
476306,208.990283,541,1153,110700,42.285123,-83.801513,66.320000,30.0,0.0,0.0,-14.0,59.024395,376.5,3500


In [4]:
#Para ahorrar RAM
del phev
del hev
del ice

Creamos índice separador

In [5]:
def create_idx(data):
  i = 0
  j = 0
  indx = []
  idx = 0
  prevlat = data['Latitude[deg]'][0]
  prevlon = data['Longitude[deg]'][0]
  prevtrip = data['Trip'][0]
  pendiente = False
  for trip,lat,lon in zip(data['Trip'], data['Latitude[deg]'], data['Longitude[deg]']):
    if pendiente:
      idx += 1
      pendiente = False
    if prevlat!=lat or prevlon!=lon or prevtrip!=trip:
      pendiente = True
    indx.append(idx)
    prevlat = lat
    prevlon = lon
    prevtrip = trip
  return indx

indices = create_idx(ev)
ev.insert(len(ev.columns),'Indx',indices)

Creamos variable de potencia instántanea

In [6]:
def suma_riemann(data):
  potencia = []
  previdx = indices[0]
  prevtime = data['Timestamp(ms)'][0]
  pot = 0
  for time,curr,volt,idx in zip(data['Timestamp(ms)'][1:],data['HV Battery Current[A]'][1:],data['HV Battery Voltage[V]'][1:],data['Indx'][1:]):
    if previdx != idx:
      potencia.append(pot)
      pot = 0
    else:
      pot += (time-prevtime)*curr*volt/3600
    previdx = idx
    prevtime = time
  potencia.append(pot) 
  return pd.DataFrame(potencia, columns = ['Potencia instantanea'])

potencia = suma_riemann(ev)

Creamos variable de aceleración

In [7]:
def acceleration(data):
  acel = []
  previdx = indices[0]
  prevtime = data['Timestamp(ms)'][0]
  prevspeed = data['Vehicle Speed[km/h]'][0]
  temp = []
  for time,speed,idx in zip(data['Timestamp(ms)'][1:],data['Vehicle Speed[km/h]'][1:],data['Indx'][1:]):
    if previdx != idx:
      acel.append(np.mean(temp))
      temp = []
    else:
      temp.append((speed-prevspeed)/(time-prevtime))
    previdx = idx
    prevspeed = speed
    prevtime = time
  acel.append(np.mean(temp)) 
  return pd.DataFrame(acel, columns = ['Mean accel'])

acel = acceleration(ev)

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Creamos otras variables con la función groupby de Python

In [10]:
means = ev.groupby(ev['Indx']).mean().loc[:,['OAT[DegC]','Vehicle Speed[km/h]']]
vars = ev.groupby(ev['Indx']).var().loc[:,['Vehicle Speed[km/h]']]
firsts = ev.groupby(ev['Indx']).first().loc[:,['Latitude[deg]','Longitude[deg]']]
lasts = ev.groupby(ev['Indx']).last().loc[:,['Latitude[deg]','Longitude[deg]']]

vars.columns = ['Variance speed']
means.columns = ['Mean OAT', 'Mean speed']
firsts.columns = ['First lat', 'First lon']
lasts.columns = ['Last lat', 'Last lon']
new_df = pd.concat([means,vars,potencia,acel,firsts,lasts], axis=1)

Creamos variable de distancia

In [11]:
def dists(data):
  dists = []
  for prevlat,prevlon,lat,lon in zip(data['First lat'],data['First lon'],data['Last lat'], data['Last lon']):
    dists.append(geopy.distance.geodesic((prevlat,prevlon),(lat,lon)).m)
  return pd.DataFrame(dists, columns = ['Distance'])
distancia = dists(new_df)
new_df = pd.concat([new_df,distancia],axis=1)

Calculamos la elevación

In [12]:
from re import T
def funcion(df1, API_endpoint, inicio, fin):
    data=[]
    for i in range(inicio,fin):
        data.append([df1['First lat'][i],df1['First lon'][i]])
    x = 0
    while x != 200:
        r = requests.post(url = API_endpoint, json= data)
        x = r.status_code
    str = r.content.decode("utf-8")
    str = str.replace('[','')
    str = str.replace(']','')
    lista = list(np.float_(str.split(',')))
    for i in range (inicio,fin):
        df1['elevation_2'][i]=lista[i-inicio]
    return(df1)

API_ENDPOINT = "https://elevation.racemap.com/api"
new_df.insert(len(new_df.columns),'elevation_2',[np.nan]*len(new_df))
elev_batch1 = funcion(new_df,API_ENDPOINT,0,9000)
elev_batch2 = funcion(new_df,API_ENDPOINT,9000,18000)
elev_batch3 = funcion(new_df,API_ENDPOINT,18000,27000)
elev_batch4 = funcion(new_df,API_ENDPOINT,27000,36000)
elev_batch5 = funcion(new_df,API_ENDPOINT,36000,45000)
elev_batch6 = funcion(new_df,API_ENDPOINT,45000,54000)
elev_batch7 = funcion(new_df,API_ENDPOINT,54000,63000)
new_df = funcion(new_df,API_ENDPOINT,63000,len(new_df))

Creamos variable de pendiente

In [13]:
def slope(data):
  slope = []
  prevelev = data['elevation_2'][0]
  for elev in data['elevation_2'][1:]:
    slope.append(elev-prevelev)
    prevelev = elev
  slope.append(0)
  return pd.DataFrame(slope, columns = ['Slope'])
pendiente = slope(new_df)
new_df = pd.concat([new_df,pendiente],axis = 1)
new_df.drop(columns='elevation_2',inplace=True)

Limpiar datos

In [14]:
new_df = new_df.dropna()
new_df.drop(columns = ['First lat'], axis =1, inplace=True)
new_df.drop(columns = ['First lon'], axis =1, inplace=True)
new_df.drop(columns = ['Last lat'], axis =1, inplace=True)
new_df.drop(columns = ['Last lon'], axis =1, inplace=True)
new_df.to_csv('content\out.csv')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
new_df

,Mean OAT,Mean speed,Variance speed,Potencia instantanea,Mean accel,Distance,Slope
0,5.0,51.694999,1.984315,6920.236111,-0.001051,76.492967,-2.335269
1,5.0,54.172499,0.245163,-9701.500000,-0.000046,75.549283,-0.204605
2,5.0,52.019998,0.448600,-725.625000,-0.000727,68.950100,-0.604567
3,5.0,53.641428,0.046214,-10736.562500,0.000216,69.942232,-0.515998
4,5.0,49.864999,13.726877,-8320.152778,-0.001749,72.945921,-1.482000
...,...,...,...,...,...,...,...
66135,30.0,34.353999,29.512179,-19430.666667,0.016881,28.023110,-0.320146
66136,30.0,47.181999,12.418866,-23400.972222,0.002369,39.867951,-0.490000
66137,30.0,57.470998,8.687869,-23565.125000,0.001607,63.876982,0.000000
66138,30.0,65.121425,2.091379,-11531.291667,0.000583,52.974001,0.000000


Entrenamiento de modelos

In [19]:
from sklearn.linear_model import LinearRegression, QuantileRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model, svm
y = new_df['Potencia instantanea']
X = new_df.drop(columns='Potencia instantanea')
X_train, X_test, y_train, y_test = train_test_split(X, y)
reg = LinearRegression().fit(X_train, y_train)
print('Linear Regression Score: ' , reg.score(X_train, y_train))
predicted = reg.predict(X_test)
print('Linear Regression MSE: ' , mean_squared_error(y_test, predicted))
reg = linear_model.Ridge(alpha=.5).fit(X_train,y_train)
print('Ridge Regression Score: ', reg.score(X_train,y_train))
predicted = reg.predict(X_test)
print('Ridge Regression MSE: ' , mean_squared_error(y_test, predicted))
reg = svm.SVR()
reg.fit(X_train, y_train)
print('SVM Regression Score: ', reg.score(X_train,y_train))
predicted = reg.predict(X_test)
print('SVM Regression MSE: ' , mean_squared_error(y_test, predicted))

Linear Regression Score:  0.16712349364757384
Linear Regression MSE:  62077197657.94985
Ridge Regression Score:  0.16705749546123883
Ridge Regression MSE:  62089708746.98166
SVM Regression Score:  -0.0017140635203030108
SVM Regression MSE:  76733591875.99452


In [20]:
corrm = new_df.corr()
corrm

,Mean OAT,Mean speed,Variance speed,Potencia instantanea,Mean accel,Distance,Slope
Mean OAT,1.000000,0.049675,0.009765,-0.002884,0.004766,0.002192,-0.001583
Mean speed,0.049675,1.000000,-0.215820,-0.016352,-0.013187,0.044350,-0.014792
Variance speed,0.009765,-0.215820,1.000000,0.199202,-0.037495,0.342258,0.019876
Potencia instantanea,-0.002884,-0.016352,0.199202,1.000000,-0.011020,0.405449,0.077346
Mean accel,0.004766,-0.013187,-0.037495,-0.011020,1.000000,0.016540,-0.036622
Distance,0.002192,0.044350,0.342258,0.405449,0.016540,1.000000,0.038421
Slope,-0.001583,-0.014792,0.019876,0.077346,-0.036622,0.038421,1.000000


In [8]:
'''
def aggregate(data):
  dist = []
  info_elev = []
  mean_speed = []
  median_speed = []
  var_speed = []
  mean_OAT = []
  mean_acel = []
  i = 0
  j = 0
  while i < len(data):
    temp_speed = []
    temp_OAT = []
    temp_acel = []
    lat_i = float(data.iloc[i,4])
    lon_i = float(data.iloc[i,5])
    lat_j = float(data.iloc[j,4])
    lon_j = float(data.iloc[j,5])
    while j < len(data) and (lat_i==lat_j and lon_i==lon_j):
      lat_j = float(data.iloc[j,4])
      lon_j = float(data.iloc[j,5])
      temp_speed.append(ev.iloc[j,6])
      temp_OAT.append(ev.iloc[j,7])
      if j+1<len(data):
        temp_acel.append((ev.iloc[j+1,6]-ev.iloc[j,6])/(ev.iloc[j+1,3]-ev.iloc[j,3]))
      j += 1
    if ev.iloc[j-1,2] != ev.iloc[i,2]: #Hay cambio de trip
      temp_OAT.pop()
      temp_speed.pop()
      temp_acel.pop()
      lat_j = float(data.iloc[j-2,4])
      lon_j = float(data.iloc[j-2,5]) 
    dist.append(geopy.distance.geodesic((lat_i,lon_i),(lat_j,lon_j)).m)
    info_elev.append([lat_i,lon_i,lat_j,lon_j])
    mean_OAT.append(np.mean(temp_OAT))
    mean_speed.append(np.mean(temp_speed))
    var_speed.append(np.var(temp_speed))
    #median_speed.append(np.median(temp_speed)) #Lo comentamos porque sacar la mediana es demorado y tiene colinealidad con el mean
    mean_acel.append(np.mean(temp_acel))
    i = j + 1
  return dist, info_elev, mean_speed, median_speed, var_speed, mean_OAT, mean_acel
dist, info_elev, mean_speed, median_speed, var_speed, mean_OAT, mean_acel = aggregate(ev)
'''

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyboardInterrupt: ignored

In [7]:
'''
def suma_riemann(data):
  potencia = []
  i = 0
  prevlat = None
  prevlon = None
  while i < len(data):
    agreg = 0
    lat = float(data.iloc[i,4])
    lon = float(data.iloc[i,5])
    while i < len(data) and (not prevlat or (lat==prevlat and lon==prevlon)):
      agreg += (data.iloc[i,3]-data.iloc[i-1,3])*data.iloc[i,10]*data.iloc[i,12]
      prevlat = lat
      prevlon = lon
      lat = float(data.iloc[i,4])
      lon = float(data.iloc[i,5])
      i += 1
    if i>=len(data):
      potencia.append(agreg)
      break
    agreg += (data.iloc[i,3]-data.iloc[i-1,3])*data.iloc[i,10]*data.iloc[i,12]/1000
    potencia.append(agreg)
    prevlat = lat
    prevlon = lon
    i += 1
  return potencia

potencia = suma_riemann(ev)
'''